# ANU ASTR4004 2024 - Week 8 (24+26 September 2024)

Author: Sven Buder (sven.buder@anu.edu.au)

Based on the tutorial by Yuan-Sen Ting from ASTR4004 2023

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Our-data:-APOGEE-DR17-and-GALAH-DR4" data-toc-modified-id="Our-data:-APOGEE-DR17-and-GALAH-DR4-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Our data: APOGEE DR17 and GALAH DR4</a></span></li><li><span><a href="#k-means-with-APOGEE-DR17" data-toc-modified-id="k-means-with-APOGEE-DR17-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>k-means with APOGEE DR17</a></span><ul class="toc-item"><li><span><a href="#Initializing-K-means-Clustering-Parameters" data-toc-modified-id="Initializing-K-means-Clustering-Parameters-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Initializing K-means Clustering Parameters</a></span></li><li><span><a href="#Visualizing-the-Data-and-Cluster-Centroids" data-toc-modified-id="Visualizing-the-Data-and-Cluster-Centroids-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Visualizing the Data and Cluster Centroids</a></span></li><li><span><a href="#Optimizing-the-K-means-Clustering" data-toc-modified-id="Optimizing-the-K-means-Clustering-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Optimizing the K-means Clustering</a></span></li><li><span><a href="#Evaluating-the-K-means-Model" data-toc-modified-id="Evaluating-the-K-means-Model-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Evaluating the K-means Model</a></span></li><li><span><a href="#Monitoring-Convergence-in-K-means" data-toc-modified-id="Monitoring-Convergence-in-K-means-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Monitoring Convergence in K-means</a></span></li><li><span><a href="#Visualising-EM" data-toc-modified-id="Visualising-EM-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Visualising EM</a></span></li><li><span><a href="#Why-May-K-means-Not-Work-Well?" data-toc-modified-id="Why-May-K-means-Not-Work-Well?-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Why May K-means Not Work Well?</a></span></li></ul></li><li><span><a href="#Gaussian-Mixture-Models-with-APOGEE-DR17" data-toc-modified-id="Gaussian-Mixture-Models-with-APOGEE-DR17-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Gaussian Mixture Models with APOGEE DR17</a></span><ul class="toc-item"><li><span><a href="#Linear-and-Log-Likelihood-function" data-toc-modified-id="Linear-and-Log-Likelihood-function-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Linear and Log-Likelihood function</a></span></li><li><span><a href="#Maximizing-Log-Likelihood-Through-EM-Algorithm" data-toc-modified-id="Maximizing-Log-Likelihood-Through-EM-Algorithm-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Maximizing Log-Likelihood Through EM Algorithm</a></span></li><li><span><a href="#Initializing-Gaussian-Mixture-Model-Parameters" data-toc-modified-id="Initializing-Gaussian-Mixture-Model-Parameters-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Initializing Gaussian Mixture Model Parameters</a></span></li><li><span><a href="#Visualizing-the-Data-and-Gaussian-Components" data-toc-modified-id="Visualizing-the-Data-and-Gaussian-Components-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Visualizing the Data and Gaussian Components</a></span></li><li><span><a href="#Optimizing-the-Gaussian-Mixture-Model-with-EM" data-toc-modified-id="Optimizing-the-Gaussian-Mixture-Model-with-EM-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Optimizing the Gaussian Mixture Model with EM</a></span></li><li><span><a href="#Evaluating-the-Model" data-toc-modified-id="Evaluating-the-Model-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Evaluating the Model</a></span></li><li><span><a href="#Off-the-shelf-GMM" data-toc-modified-id="Off-the-shelf-GMM-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Off-the-shelf GMM</a></span></li><li><span><a href="#How-many-components-do-we-need?" data-toc-modified-id="How-many-components-do-we-need?-3.8"><span class="toc-item-num">3.8&nbsp;&nbsp;</span>How many components do we need?</a></span></li></ul></li><li><span><a href="#Interactive:-GALAH-DR4-instead-of-APOGEE-DR17" data-toc-modified-id="Interactive:-GALAH-DR4-instead-of-APOGEE-DR17-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Interactive: GALAH DR4 instead of APOGEE DR17</a></span></li></ul></div>

In [ ]:
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

import numpy as np
from astropy.table import Table

# We will use these later for the Gaussian Mixture Models
from scipy.stats import multivariate_normal as mvnorm
from scipy.stats import chi2

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib.colors import LogNorm

# Make the size and fonts larger for this presentation
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 12

## Our data: APOGEE DR17 and GALAH DR4

In [ ]:
try:
    np.random.seed(4)
    
    # Read in GALAH DR4
    galah_dr4 = Table.read('data/galah_dr4_allstar_240705.fits')

    # Apply quality cuts
    select_galah = (
        (galah_dr4['flag_sp'] == 0) & (galah_dr4['flag_mg_fe'] == 0) & 
        (galah_dr4['fe_h'] > -1.0) & (galah_dr4['fe_h'] < 0.75) &
        (galah_dr4['mg_fe'] > -0.35) & (galah_dr4['mg_fe'] < 0.65)
    )
    galah_dr4 = galah_dr4[['fe_h','mg_fe']][select_galah]

    # Only for this tutorial: select only 50,000 randomly drawn entries
    random_indices = np.random.choice(len(galah_dr4['fe_h']), size=50000, replace=False)
    galah_dr4 = galah_dr4[random_indices]

    # Save data
    galah_dr4.write('data/galah_dr4_feh_mgfe.fits', overwrite=True)

except:
    galah_dr4 = Table.read('data/galah_dr4_feh_mgfe.fits')

In [ ]:
try:
    np.random.seed(17)

    # Read in APOGEE DR17
    apogee_dr17 = Table.read('data/allStar-dr17-synspec_rev1.fits',1)

    # Apply quality cuts
    select_apogee = (
        # No major flag
        (apogee_dr17['ASPCAPFLAG'] == 0) &
        # No flag for abundances
        (apogee_dr17['FE_H_FLAG'] == 0) & (apogee_dr17['MG_FE_FLAG'] == 0) &
        # Select disk stars
        (apogee_dr17['FE_H'] > -1.0) & (apogee_dr17['FE_H'] < 0.75) &
        (apogee_dr17['MG_FE'] > -0.35) & (apogee_dr17['MG_FE'] < 0.65) &
        (apogee_dr17['MG_FE'] + apogee_dr17['FE_H'] > -0.75) &
        # Select giants
        (apogee_dr17['TEFF'] < 5500) & (apogee_dr17['LOGG'] < 3.5)
    )
    apogee_dr17 = apogee_dr17[['FE_H','MG_FE']][select_apogee]

    # Only for this tutorial: select only 50,000 randomly drawn entries
    random_indices = np.random.choice(len(apogee_dr17['FE_H']), size=50000, replace=False)
    apogee_dr17 = apogee_dr17[random_indices]

    # Save data
    apogee_dr17.write('data/apogee_dr17_feh_mgfe.fits', overwrite=True)

except:
    apogee_dr17 = Table.read('data/apogee_dr17_feh_mgfe.fits')

In [ ]:
# Let's plot the [Fe/H] vs. [Mg/Fe] diagrams for APOGEE DR17 and GALAH DR4

f, gs = plt.subplots(1,2,figsize=(10,3.5),sharex=True,sharey=True)

ax = gs[0]
ax.set_xlabel('[Fe/H]')
ax.set_ylabel('[Mg/Fe')
ax.set_title('APOGEE DR17')
h = ax.hist2d(
    apogee_dr17['FE_H'],
    apogee_dr17['MG_FE'],
    bins = 100,
    norm = LogNorm()
)
cbar = plt.colorbar(h[-1], ax=ax, label = 'Nr.')

ax = gs[1]
ax.set_xlabel('[Fe/H]')
ax.set_title('GALAH DR4')
h = ax.hist2d(
    galah_dr4['fe_h'],
    galah_dr4['mg_fe'],
    bins = 100,
    norm = LogNorm()
)
cbar = plt.colorbar(h[-1], ax=ax, label = 'Nr.')

plt.tight_layout()

## k-means with APOGEE DR17

In [ ]:
# for this tutorial, let's use variables rather than keywords
feh  = np.array(apogee_dr17['FE_H'])
mgfe = np.array(apogee_dr17['MG_FE'])

In [ ]:
# noramlize data
feh = (feh - np.mean(feh)) / np.std(feh)
mgfe = (mgfe - np.mean(mgfe)) / np.std(mgfe)

# prepare data for sklearn
data = np.vstack([feh, mgfe]).T
data_labels = ["[Fe/H]", "[Mg/Fe]"]

### Initializing K-means Clustering Parameters

Before running the K-means algorithm on our dataset, it's important to set initial values for the model's centroids. These initializations guide the iterative process of the K-means algorithm. For our problem, we'll use a two-cluster K-means model to capture the bimodal distribution of alpha-enriched and alpha-normal stars.

We will initialize the centroids of our two clusters as follows:

- For the first cluster ($ C_1 $):
$$
\begin{bmatrix}
    -1 \\
    +1
\end{bmatrix}
$$

- For the second cluster ($ C_2 $):
$$
\begin{bmatrix}
    +1 \\
    -1
\end{bmatrix}
$$

By initializing these parameters, we can now proceed with the K-means algorithm to cluster our dataset based on elemental abundances.


In [ ]:
# K-means initial centroids
C_0 = np.array([[-1.0, 1.0], [1.0, -1.0]])

### Visualizing the Data and Cluster Centroids

To better understand the iterative process of the K-means algorithm and to visually inspect how well the centroids represent different clusters in the dataset, we plot both the data points and the centroids. Optionally, we can also visualize the "redness" of each data point, which could represent its closeness to a specific centroid or its likelihood of belonging to a particular cluster.

**Helper Functions for Plotting: plot_data_and_centroids Function**

The `plot_data_and_centroids` function serves the primary role of plotting both the data and the centroids on the same chart. Here are the parameters:

- `data`: The actual data points you wish to plot.
- `centroids`: The positions of the centroids.
- `redness`: Optional parameter to color data points based on some metric, which could be their distance to the nearest centroid or some other relevant measure.

This function uses an optional 'redness' parameter to color the data points. When the 'redness' values are provided, each point will be colored on a blue-to-red scale based on this value. This visual cue can help us better understand the cluster assignment at each step of the K-means iteration.

**Optional: Custom Color Map**

A custom color map (`br_cmap`) is defined using Matplotlib's `LinearSegmentedColormap`. This color map is used when coloring data points based on their 'redness' value. The color map transitions from blue to red, allowing for a clear visualization of cluster memberships or distances to centroids.

These helper functions enable us to visually assess how well the K-means algorithm is performing at each iteration and offer insights into the clustering process.


In [ ]:
def plot_data_and_centroids(data, centroids, redness=None):
    plt.figure(figsize=(8,8))
    
    br_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("seismic",["b","r"])
    
    # Plotting the data points
    if redness is not None:
        assert len(redness) == data.shape[0]
        assert all(_ >= 0 and _ <= 1 for _ in redness)
        c = redness
    else:
        c = 'grey'
    plt.scatter(data[:, 0], data[:, 1], c=c, s=0.1, alpha=0.3, linewidths=2, cmap=br_cmap, label="Data points")
    
    # Plotting the centroids with different colors
    centroid_colors = ['b', 'r']
    for i in range(centroids.shape[0]):
        plt.scatter(centroids[i, 0], centroids[i, 1], c=centroid_colors[i], marker='x', s=100, label=f"Centroid {i+1}")
    
    plt.xlabel("[Fe/H]")
    plt.ylabel("[Mg/Fe]")
    plt.legend()
    plt.show()

# Assuming data and initial centroids C_0 are defined earlier, and that 'redness' values are calculated somehow
plot_data_and_centroids(data, C_0)

### Optimizing the K-means Clustering

As discussed in lectures, the K-means algorithm can be optimized through an iterative approach. This involves two steps: the assignment step, and the update step.

**Function Signatures**

The primary functions involved in K-means are:

- For the E-step or Assignment step,
```python
def assignment_step(X, centroids):
```

- For the M-step or the Update step,
```python
def update_step(X, labels):
```

**Tips**

You can make use of the `scipy.spatial.distance` library to compute Euclidean distances between points:

```python
from scipy.spatial import distance
```

You can use `distance.euclidean(a, b)` to compute the distance between points `a` and `b`.

In this K-means implementation:

1. `assignment_step`: Assigns each data point to the nearest centroid, effectively partitioning the data into clusters.
2. `update_step`: Calculates the new centroids by taking the mean of all the data points in each cluster.

Unlike GMM, K-means doesn't work with covariance matrices, Gaussian components, or weighted probabilities. It's a simpler algorithm in that regard.

In [ ]:
from scipy.spatial import distance
import numpy as np

def assignment_step(X, centroids):
    """
    Performs the E-step of the K-means algorithm.
    """
    labels = np.argmin([distance.cdist(X, c.reshape(1, -1), 'euclidean') for c in centroids], axis=0)
    return labels.flatten()

def update_step(X, labels, K):
    """
    Performs the M-step of the K-means algorithm.
    """
    new_centroids = np.array([X[labels == k].mean(axis=0) for k in range(K)])
    return new_centroids

### Evaluating the K-means Model

To evaluate the performance of the K-means model, you can calculate its inertia.

The function computes the inertia by summing up the squared Euclidean distances between each data point and the centroid of its cluster. The resulting value will be a single scalar that measures how well the centroids fit the data $ \mathbf{X} $.


In [ ]:
from scipy.spatial import distance
import numpy as np

def calculate_inertia(X, labels, centroids):
    inertia = 0
    for i in range(len(centroids)):
        cluster_points = X[labels == i]
        inertia += np.sum([distance.euclidean(p, centroids[i]) ** 2 for p in cluster_points])
    return inertia

### Monitoring Convergence in K-means

To check if the K-means algorithm is converging, one can track the model's inertia against the number of iterations. The idea is to run K-means for a fixed number of iterations and compute the inertia at each step. This information can then be visualized to assess the algorithm's convergence.



In [ ]:
# Assuming `data` is your input data
# Initialize centroids
initial_centroids = C_0  

# Number of clusters
K = 2

# Number of iterations
iterations = 100

# Array to hold the inertia values
inertia_values = np.zeros(iterations)

# Initialize centroids to some starting values (C_0)
centroids = initial_centroids

# Run the K-means algorithm
for i in range(iterations):
    # Perform the Assignment step to update labels
    labels = assignment_step(data, centroids)
    
    # Compute the inertia of the current model
    inertia_values[i] = calculate_inertia(data, labels, centroids)
    
    # Update centroids for the next iteration
    centroids = update_step(data, labels, K)

# Plotting the inertia values from iteration 0
plt.title("Inertia Values from Iteration 0")
plt.xlabel("Number of Updates")
plt.ylabel("Inertia")
plt.plot(inertia_values)
plt.show()

# Plotting the inertia values starting from iteration 1
plt.title("Inertia Values from Iteration 1")
plt.xlabel("Number of Updates")
plt.ylabel("Inertia")
plt.plot(inertia_values[1:])
plt.show()

### Visualising EM

Use the function `plot_data_and_centroids` to visualize the data points and your current cluster centroids at different stages of the K-means algorithm.


In [ ]:
# Initialize centroids to some starting values (C_0)
centroids = C_0 

# Number of clusters
K = 2

# Number of iterations for the K-means algorithm
iterations = 50

# Array to keep track of the inertia values
inertia_values = np.zeros(iterations)

# Run the K-means algorithm
for i in range(iterations):
    
    # Perform the Assignment step to update labels
    labels = assignment_step(data, centroids)
    
    # Compute the inertia of the current model
    inertia_values[i] = calculate_inertia(data, labels, centroids)
    
    # Visualize the model every 5 iterations
    if i % 5 == 0:
        print(f"Iteration {i}: Current Inertia = {inertia_values[i]}")
        
        # Show the current state of the model
        plot_data_and_centroids(data, centroids, redness=labels)
        
    # Perform the Update step to update centroids
    centroids = update_step(data, labels, K)

### Why May K-means Not Work Well?

In our analysis, we observed that the K-means clustering algorithm did not perform well with our elemental abundances data that forms a double moon shape. The reason lies in the assumptions behind the K-means algorithm, particularly its use of Euclidean distance to partition data points into clusters. K-means tries to minimize the variance within each cluster, which is equivalent to minimizing the Euclidean distance from each data point to its cluster's centroid. This works well for clusters that are spherical and equally sized, but not for data with more complex shapes or varying densities.

**Limitations of K-means:**

1. **Spherical Assumption**: K-means assumes that the clusters are spherical and equally sized, which doesn't hold true for double moon-shaped data.
  
2. **Euclidean Distance**: The algorithm uses Euclidean distance to allocate points to the nearest cluster. For more complex shapes like double moons, a different distance measure or model might be more appropriate.
  
3. **Lack of Flexibility**: K-means does not have the flexibility to account for different shapes or orientations of clusters.

To overcome these limitations, we turn to Gaussian Mixture Models (GMMs), a more flexible approach that can model elliptical clusters and is capable of handling different cluster shapes and orientations. Unlike K-means, GMMs do not assume equal-sized or spherical clusters, and they provide a probabilistic framework to capture uncertainty, making them a more suitable option for clustering our double moon-shaped data.


## Gaussian Mixture Models with APOGEE DR17

In [ ]:
# for this tutorial, let's again use variables rather than keywords
feh  = np.array(apogee_dr17['FE_H'])
mgfe = np.array(apogee_dr17['MG_FE'])

data = np.vstack([feh, mgfe]).T
data_labels = ["[Fe/H]", "[Mg/Fe]"]

Gaussian Mixture Models (GMMs) offer a robust approach to model complex data distributions. When given a dataset $\{\mathbf{x}_1, \ldots, \mathbf{x}_n \}$ with each $\mathbf{x}_i \in \mathbb{R}^D$, GMMs postulate that this data is generated from a sum of $K$ different Gaussian distributions.

We want to find $K$ components, each with a mixture weight $\pi_k$ and individual Gaussian distributions $\mathcal{N}(x \mid \mu_k, \Sigma_k )$ with means $\mu_k$ and covariances $\Sigma_k$. The mixture weights must satisfy the constraint:

$$
\sum_{k=1}^K \pi_k = 1
$$

The goal is to find $\pi$, $\mu$, and $\Sigma$ that maximize this function. However, this is easier said than done due to the complex inter-dependencies between these parameters.

### Linear and Log-Likelihood function

Imagine drawing a random number $p$ uniformly from the interval $[0,1)$. Based on the value of $p$, we determine which Gaussian distribution to sample from. For example, if $p$ lies in the range $[\sum_{i=1}^{k-1} \pi_i, \sum_{i=1}^{k} \pi_k)$, we sample $\mathbf{x}$ from $\mathcal{N}(x \mid \mu_k, \Sigma_k )$.

Mathematically, we can then describe the probability distribution $p$:

$$
p(\mathbf{x}) := \sum_{k=1}^K \pi_k \mathcal{N}(x \mid \mu_k, \Sigma_k )
$$

The log-likelihood function for GMMs is expressed as:

$$
\log p(\mathbf{x} \mid \pi, \mu, \Sigma) 
= \sum_{n=1}^N \log \left\{ \sum_{k=1}^K \pi_k \mathcal{N}(\mathbf{x} \mid \mu_k, \Sigma_k) \right\}
$$

### Maximizing Log-Likelihood Through EM Algorithm

**Responsibilities and Effective Number of Points**

We introduce a variable called the 'responsibility', denoted by $\gamma_{nk}$, which essentially quantifies the likelihood that the $n^{th}$ data point is generated by the $k^{th}$ Gaussian. It is computed as:

$$
\gamma_{nk} = \frac{\pi_{k} \mathcal{N}( \mathbf{x}_n \mid \mu_k, \Sigma_k ) }
    { \sum_{k=1}^K \pi_{k} \mathcal{N}( \mathbf{x}_n \mid \mu_k, \Sigma_k)}
$$

We then define $N_k$, the effective number of points for the $k^{th}$ Gaussian, as:

$$
N_k := \sum_{n=1}^N \gamma_{nk}
$$

**EM Algorithm Steps**

The Expectation-Maximization (EM) algorithm proceeds in a series of steps to iteratively update the model parameters and maximize the log-likelihood.

- **Step 1 (Initialization)**: Choose initial values for $\mu_k, \Sigma_k, \pi_k$, and calculate the initial log-likelihood.

- **Step 2 (Expectation)**: Compute the responsibilities using the current parameters.

- **Step 3 (Maximization)**: Update the parameters based on the newly computed responsibilities:

\begin{align*}
N_k^\text{new} & := \sum_{n=1}^N \gamma_{nk} \\
\pi_k^{\text{new}} & := \frac{N_k^\text{new}}{N} \\
\mu_k^\text{new} & := \frac{1}{N_k^\text{new}} \sum_{n=1}^N \gamma_{nk} \mathbf{x}_n \\
\Sigma_k^\text{new} & := \frac{1}{N_k^\text{new}} \sum_{n=1}^N \gamma_{nk} (\mathbf{x}_n - \mu_k^\text{new}) (\mathbf{x}_n - \mu_k^\text{new})^T
\end{align*}


- **Step 4 (Evaluate)**: Calculate the new log-likelihood. If it hasn't converged, return to Step 2.

By iteratively applying these steps, the EM algorithm arrives at an optimized set of parameters that maximize the log-likelihood.

### Initializing Gaussian Mixture Model Parameters

Before running the Gaussian Mixture Model (GMM) on our dataset, it's crucial to set initial values for the model's parameters. These initializations guide the iterative process of the Expectation-Maximization (EM) algorithm, which we will be using to optimize the model. For our problem, we'll use a two-component Gaussian Mixture Model to capture the bimodal distribution of alpha-enriched and alpha-normal stars.

**Initial Means: $ \mu_0 $**

We will initialize the means of our two Gaussian components as follows:

- For the first Gaussian ($ \mu_1 $):
$$
\begin{bmatrix}
    -1 \\
    +1
\end{bmatrix}
$$

- For the second Gaussian ($ \mu_2 $):
$$
\begin{bmatrix}
    +1 \\
    -1
\end{bmatrix}
$$

**Initial Covariance Matrices: $ \Sigma_0 $**

Both covariance matrices will be initialized as identity matrices. This assumes, initially, that the features are uncorrelated and have a unit variance.

**Initial Mixture Weights: $ \pi_0 $**

Initially, we will assume that both Gaussian components are equally likely to generate any given data point. Thus, the initial mixture weights will be equal; specifically, each will be set to 0.5.

**Summary of Initial Parameters**

- `mu_0`: A $2 \times 2$ matrix containing the initial means of the Gaussians.
  - $\mu_0 = \left[ \begin{array}{cc} -1 & 1 \\ 1 & -1 \end{array} \right]$

- `Sigma_0`: A $2 \times 2 \times 2$ 3-tensor containing the initial covariance matrices.
  - $\Sigma_0 = \left[ \begin{array}{cc} 1 & 0 \\ 0 & 1 \end{array} \right] \times 2$

- `pi_0`: A vector containing the initial mixture weights.
  - $\pi_0 = \left[ 0.5, 0.5 \right]$

By initializing these parameters, we can now proceed with the EM algorithm to fit the Gaussian Mixture Model to our dataset.

In [ ]:
# Solution
pi_0 = np.array([0.5,0.5])
mu_0 = np.array([[-1.0,1.0],[1.0,-1.0]])
Sigma_0 = np.array([ [[1.0,0.0],[0.0,1.0]] , [[1.0,0.0],[0.0,1.0]]])

### Visualizing the Data and Gaussian Components

To better understand the Expectation-Maximization (EM) process and the distribution of our data, we will plot both the data points and ellipses representing our Gaussian components. The ellipses will provide a visual interpretation of the covariance and mean of each Gaussian in our mixture model.

**plot_cov_ellipse Function**

The `plot_cov_ellipse` function plots an ellipse based on a given 2x2 covariance matrix and a location for the ellipse's center. The function also allows the customization of the ellipse's appearance through various parameters.

- `cov`: The 2x2 covariance matrix to base the ellipse on.
- `pos`: The location of the center of the ellipse.
- `volume`: The volume inside the ellipse; default is 0.5.
- `ax`: The axis to plot the ellipse on; defaults to the current axis.

**plot_components Function**

The `plot_components` function uses `plot_cov_ellipse` to plot ellipses for each Gaussian component in our mixture model. It takes in the means (`mu`) and covariances (`Sigma`) of each Gaussian, along with their respective colors, to render these ellipses on the plot.

**plot_data Function**

The `plot_data` function plots the actual data points. Optionally, it can color the points based on a 'redness' value, allowing us to visually separate data that might belong to different components of the Gaussian mixture.

These helper functions will enable us to visualize the Gaussian components and their evolution as we proceed with the EM algorithm.

In [ ]:
# plot_cov_ellipse was taken from here:
# http://www.nhsilbert.net/source/2014/06/bivariate-normal-ellipse-plotting-in-python/

def plot_cov_ellipse(cov, pos, volume=.5, ax=None, fc='none', ec=[0,0,0], a=1, lw=2):
    """
    Plots an ellipse enclosing *volume* based on the specified covariance
    matrix (*cov*) and location (*pos*). Additional keyword arguments are passed on to the 
    ellipse patch artist.

    Parameters
    ----------
        cov : The 2x2 covariance matrix to base the ellipse on
        pos : The location of the center of the ellipse. Expects a 2-element
            sequence of [x0, y0].
        volume : The volume inside the ellipse; defaults to 0.5
        ax : The axis that the ellipse will be plotted on. Defaults to the 
            current axis.
    """
    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:,order]

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))

    kwrg = {'facecolor':fc, 'edgecolor':ec, 'alpha':a, 'linewidth':lw}

    # Width and height are "full" widths, not radius
    width, height = 2 * np.sqrt(chi2.ppf(volume,2)) * np.sqrt(vals)
    ellip = patches.Ellipse(xy=pos, width=width, height=height, angle=theta, **kwrg)

    ax.add_artist(ellip)
    

def plot_components(mu, Sigma, colours, *args, **kwargs):
    '''
    Plot ellipses for the bivariate normals with mean mu[:,i] and covariance Sigma[:,:,i]
    '''
    assert mu.shape[1] == Sigma.shape[2]
    assert mu.shape[0] == 2
    assert Sigma.shape[0] == 2
    assert Sigma.shape[1] == 2
    for i in range(mu.shape[1]):
        kwargs['ec'] = colours[i]
        plot_cov_ellipse(Sigma[i], mu[i], *args, **kwargs)

br_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("seismic",["b","r"])

def plot_data(redness=None):
    if redness is not None:
        assert len(redness) == data.shape[0]
        assert all(_ >= 0 and _ <= 1 for _ in redness)
        c = redness
    else:
        c = 'grey'
    plt.figure(figsize=(8,8))
    plt.scatter(data[:,0],data[:,1], marker='.', s=0.1, alpha=0.3, linewidths=2, c=c, cmap=br_cmap)
    plt.xlabel(data_labels[0])
    plt.ylabel(data_labels[1])
    plt.axis([-2,2,-2,2])
    plt.axis('equal')

In the next code cell, we demonstrate how to utilize the plotting functions we've defined earlier. We'll plot the dataset along with the initial guesses for our Gaussian components. This visualization will help us understand our starting point before we begin iterating with the Expectation-Maximization algorithm.

The code will only execute if the required variables (`mu_0`, `Sigma_0`, and `data`) have been previously defined. If these variables are in place, the function `plot_data()` will plot the data points, and `plot_components()` will overlay the initial Gaussian ellipses on the same plot.

This plot will serve as a reference point, helping us to evaluate how well our Gaussian Mixture Model fits the data as we proceed with the EM algorithm.

In [ ]:
# Check if the required variables are defined
if 'mu_0' in locals() and 'Sigma_0' in locals() and 'data' in locals():
    # Plot the data points
    plot_data()
    # Plot the initial Gaussian components
    plot_components(mu_0, Sigma_0, ['b', 'r'], 0.2)
    # Display the plot
    plt.show()

### Optimizing the Gaussian Mixture Model with EM

As discussed in lectures, the Gaussian Mixture Model (GMM) can be optimized using the Expectation-Maximization (EM) algorithm. This technique allows for the maximum likelihood estimation of the model parameters $\mathbf{\mu}$, $\mathbf{\Sigma}$, and $\mathbf{\pi}$.

**Function Signatures**

The primary functions involved in this algorithm are the E-step and the M-step. The suggested function signatures are as follows:

- For the E-step: 
```python
def e_step(X, mu, Sigma, pi):
```

- For the M-step:
```python
def m_step(X, gamma):
```

**Helper Functions**

In addition to these, a helper function named `weighted_normals` is used to calculate an $ N \times K $ matrix of weighted normal probabilities, i.e., $ \pi_{k} \mathcal{N}( \mathbf{x}_n \mid \mu_k, \Sigma_k) $.

The function signature for the helper function is:

```python
def weighted_normals(X, mu, Sigma, pi):
```

**Tips**

You can make use of the `scipy.stats` library to compute the multivariate normal distribution probabilities:

```python
from scipy.stats import multivariate_normal as mvnorm
```

You can use `mvnorm.pdf(x, mu, sigma)` to compute $ \mathcal{N}(\mathbf{x}_N \mid \mu_k, \Sigma_k) $.


In [ ]:
def weighted_normals(X, mu, Sigma, pi):
    """
    Calculates the numerator of the gamma_i's, i.e., the 
    weighted normal probabilities for each data point.
    """
    N, D = X.shape
    K, = pi.shape
    w_norms = np.zeros((N,K)) # (N,K)
    for k in range(K):
        w_norms[:,k] = pi[k] * mvnorm.pdf(data, mu[k], Sigma[k]) # (N,)
    return w_norms # (N,K)

def e_step(X, mu, Sigma, pi):
    """
    Performs the E-step of the EM algorithm.
    """
    w_norms = weighted_normals(X, mu, Sigma, pi)
    gamma = w_norms / np.sum(w_norms, axis=1, keepdims=True)
    return gamma

def m_step(X, gamma):
    """
    Performs the M-step of the EM algorithm.
    """
    N,D = X.shape
    _, K = gamma.shape
    Nk = gamma.sum(axis=0) # (N,K)=>(K,)
    new_pi = Nk / N # (K,)

    # Best, no iteration / sum
    new_mu = gamma.T @ X # (K,N)@(N,D) => (K,D)
    new_mu /= Nk.reshape(-1,1) # (K,D) / (K,1) => (K,D)

    # Best, no iteration / sum
    diff = X.reshape(N,1,D) - new_mu.reshape(1,K,D) # (N,1,D) - (1,K,D) => (N,K,D)
    scaled_diff = gamma.reshape(N,K,1) * diff # (N,K,1) * (N,K,D) => (N,K,D)
    new_Sigma = scaled_diff.transpose((1,2,0)) @ diff.transpose(1,0,2) # (K,D,N) @ (K,N,D) => (K,D,D)
    new_Sigma /= Nk.reshape(-1,1,1)  # (K,D,D) / (K,1,1) = (K,D,D)

    return new_mu, new_Sigma, new_pi # (K,D), (K,D,D), (K,)

### Evaluating the Model

To evaluate the performance of the Gaussian Mixture Model, we will calculate its log-likelihood for given parameters $ \mathbf{\mu} $, $ \mathbf{\Sigma} $, and $\mathbf{\pi}$.

The function computes the log-likelihood by first obtaining the weighted normal probabilities using the `weighted_normals` function. Then it sums up the log of these probabilities across all data points. The resulting value will be a single scalar that measures how well the given parameters $ \mathbf{\mu} $, $ \mathbf{\Sigma} $, and $ \mathbf{\pi} $ fit the data $ \mathbf{X} $.


In [ ]:
def log_likelihood(X, mu, Sigma, pi):
    # Calls the helper function to get the weighted normal probabilities
    w = weighted_normals(X, mu, Sigma, pi)
    
    # Computes the log-likelihood using the obtained probabilities
    ll = np.log(w.sum(axis=1)).sum()
    
    return ll

An essential aspect of any iterative algorithm, like the EM algorithm, is to check if it converges. For this purpose, we can plot the log-likelihood of the model against the number of updates made to it.

The idea is to run the EM algorithm for a fixed number of trials and calculate the log-likelihood at each step. We will then visualize this information to assess the convergence of the model.



In [ ]:
# Initialize parameters
mu = mu_0
Sigma = Sigma_0
pi = pi_0

# Number of trials
trials = 100

# Array to hold the log-likelihoods
ll = np.zeros(trials)

# Run the EM algorithm
for i in range(0, trials):
    ll[i] = log_likelihood(data, mu, Sigma, pi)
    gamma = e_step(data, mu, Sigma, pi)
    (mu, Sigma, pi) = m_step(data, gamma)

# Plotting the log-likelihoods from iteration 0
plt.title("Log-Likelihoods from Iteration 0")
plt.xlabel("Number of Updates")
plt.ylabel("Log-Likelihood")
plt.plot(ll)
plt.show()

# Plotting the log-likelihoods starting from iteration 1
plt.title("Log-Likelihoods from Iteration 1")
plt.xlabel("Number of Updates")
plt.ylabel("Log-Likelihood")
plt.plot(ll[1:])
plt.show()

In [ ]:
# Initialize parameters to some starting values (mu_0, Sigma_0, pi_0)
mu = mu_0
Sigma = Sigma_0
pi = pi_0

# Number of iterations for the EM algorithm
trials = 50

# Array to keep track of the log-likelihood values
ll = np.zeros(trials)

# Run the EM algorithm
for i in range(trials):
    # Compute the log-likelihood of the current model
    ll[i] = log_likelihood(data, mu, Sigma, pi)
    
    # Visualize the model every 5 iterations
    if i % 5 == 0:
        print(f"Iteration {i}: Current Log-Likelihood = {ll[i]}")
        
        # show the color 
        plot_data(redness = gamma[:,1])

        # Plot the Gaussian components
        plot_components(mu, Sigma, ['b', 'r'], 0.2)
        
        # Add a title to indicate the current iteration and log-likelihood
        plt.title(f"Iteration {i}: Log-Likelihood = {ll[i]}")
        plt.show()
        
    # Perform the E-step to update responsibilities
    gamma = e_step(data, mu, Sigma, pi)
    
    # Perform the M-step to update parameters
    mu, Sigma, pi = m_step(data, gamma)

### Off-the-shelf GMM

In [ ]:
from sklearn.mixture import GaussianMixture

# Set the number of components (start with 2, as expected for thin/thick disk)
gmm = GaussianMixture(
    n_components=2, 
    random_state=42, # optional, important for reproducing results
    n_init=5, # optional, but a good idea for robustness
    means_init=mu_0 # optional, but can be helpful
)
gmm.fit(data)

# Extract the predicted labels and probabilities
labels = gmm.predict(data)
probabilities = gmm.predict_proba(data)

# Plotting the [Fe/H] vs. [Mg/Fe] distribution colored by GMM components
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Left panel: [Fe/H] vs. [Mg/Fe] colored by component labels
scatter1 = ax1.scatter(data[:, 0], data[:, 1], c=labels, cmap='cividis', s=5, alpha=0.5)

# Because we are now using the off-the-shelf GMM, we have to redefine the ellipse plotting
def plot_gmm_ellipses(gmm, ax, colors=['blue', 'red']):
    for n, color in enumerate(colors):
        mean = gmm.means_[n]
        covar = gmm.covariances_[n]
        
        # Eigenvalues and eigenvectors to draw ellipses
        v, w = np.linalg.eigh(covar)
        v = np.sqrt(2.0) * np.sqrt(v)  # standard deviations
        u = w[0] / np.linalg.norm(w[0])
        
        angle = np.arctan2(u[1], u[0]) * 180 / np.pi
        ellipse = patches.Ellipse(mean, v[0], v[1], 180 + angle, color=color, alpha=0.3)
        ax.add_patch(ellipse)

plot_gmm_ellipses(gmm, ax1)  # Add ellipses for the GMM components

ax1.set_xlabel('[Fe/H]')
ax1.set_ylabel('[Mg/Fe]')
fig.colorbar(scatter1, ax=ax1, label='GMM Component', orientation='horizontal')

# Right panel: [Fe/H] vs. [Mg/Fe] colored by certainty of component assignment
certainty = np.max(probabilities, axis=1)  # Take the maximum probability (certainty) for each point
scatter2 = ax2.scatter(data[:, 0], data[:, 1], c=certainty, cmap='seismic', s=5, alpha=0.5)
ax2.set_xlabel('[Fe/H]')
ax2.set_ylabel('[Mg/Fe]')
fig.colorbar(scatter2, ax=ax2, label='Component Membership Probability', orientation='horizontal')

plt.tight_layout()
plt.show()

In [ ]:
# Weights of our GMMs
gmm.weights_

In [ ]:
# Means of our GMMs
gmm.means_

In [ ]:
# Covariances of our GMMs
gmm.covariances_

### How many components do we need?

We will discuss this in detail on Thursday, but the Bayesian and Aikake Information Criteria tell you how well your model explains the data (with a maximum Likelihood $\hat{L}$) while applying a penalty for the number of free parameters $k$.

For a given model, the AIC is defined as

\begin{equation}
\mathrm{AIC} = 2k - 2\ln\left(\hat{L}\right).
\end{equation}

BIC has a similar form but with a different penalty for the number of parameters:

\begin{equation}
\mathrm{BIC} = k\ln(n) - 2\ln\left(\hat{L}\right).
\end{equation}

The lower these values, the better.

In [ ]:
import numpy as np

n_components = np.arange(1, 11)
bics = []
aics = []

for n in n_components:
    gmm = GaussianMixture(n_components=n, random_state=42, n_init=5)
    gmm.fit(data)
    bics.append(gmm.bic(data))
    aics.append(gmm.aic(data))

# Plot BIC/AIC to find the optimal number of components
plt.figure(figsize=(8,6))
plt.plot(n_components, bics, label='BIC')
plt.plot(n_components, aics, label='AIC')
plt.xlabel('Number of Components')
plt.ylabel('BIC/AIC')
plt.legend()
plt.title('BIC and AIC for GMM with Different Components')
plt.show()

## Interactive: GALAH DR4 instead of APOGEE DR17

Apply what you have learned onto the lower precision GALAH DR4 measurements. Do you get similar results?